In [ ]:
import pandas as pd # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.feature_extraction.text import TfidfVectorizer # type: ignore
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.metrics import classification_report # type: ignore
import re
import joblib # type: ignore
import string

In [3]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [8]:
fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [7]:
true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [6]:
fake['class']=0
true['class']=1

In [10]:
data = pd.concat([fake,true], axis = 0)

In [ ]:
data.sample(10)

,text,class
16692,Outspoken British actress and columnist asks b...,0
18048,ACCRA (Reuters) - At least seven people were k...,1
17364,It s interesting that Josh Earnest still carri...,0
21315,NAIROBI (Reuters) - A United States citizen wo...,1
5005,The new head of Donald Trump s presidential ca...,0
7458,"Gay-hating, Ted Cruz-loving Pastor Kevin Swans...",0
1606,Yet another altercation has erupted on board a...,0
18741,ISTANBUL (Reuters) - Turkish prosecutors have ...,1
18774,On the eve of the upcoming 2017 White House Co...,0
9339,WASHINGTON (Reuters) - The U.S. Justice Depart...,1


In [12]:
data = data.drop(["title", "subject", "date"], axis = 1)

In [14]:
data.reset_index(inplace=True)

In [15]:
data.drop(['index'],axis=1,inplace=True)

In [16]:
data.sample(5)

,text,class
6254,Just when you thought Kentucky County Clerk Ki...,0
9411,Just another case of Love Trumps Hate LOL! Th...,0
11188,The world is going to need to make some major ...,0
43843,Fort Lauderdale and New York (Reuters) - When ...,1
29492,"MEXICO CITY, (Reuters) - A summit between U.S....",1


In [ ]:
def clean_text(text):
    text = re.sub('\[.*?\]',"",text)
    text = re.sub("\\W"," ",text)
    text = re.sub("https?:://\S+|www\.\S+","",text)
    text = re.sub('<.*?>+',"",text)
    text = re.sub('[%s]' % re.escape(string.punctuation),"",text)
    text = re.sub('\n',"",text)
    text = re.sub("\W\d\W*","",text)
    return text

In [20]:
data["text"] = data["text"].apply(clean_text)

In [21]:
x=data["text"]
y=data["class"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [22]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)

In [24]:
lr = LogisticRegression()
lr.fit(xv_train,y_train)

LogisticRegression()

In [25]:
prediction = lr.predict(xv_test)
lr.score(xv_test, y_test)

0.985924276169265

In [26]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [27]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']

In [ ]:
# Load the trained model and vectorizer
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
import joblib

vectorizer = joblib.load("vectorizer.jb")
model = joblib.load("lr_model.jb")

# Initialize Flask app
app = Flask(__name__)

@app.route('/bot', methods=['POST'])
def bot():
    incoming_msg = request.values.get('Body', '').strip()
    response = MessagingResponse()
    msg = response.message()

    if incoming_msg:
        # Clean and vectorize the input
        input_tfidf = vectorizer.transform([incoming_msg])
        prediction = model.predict(input_tfidf)[0]
        result = 'Real' if prediction == 1 else 'Fake'
        msg.body(f"The news is predicted to be: {result}")
    else:
        msg.body("Please send a valid news text.")

    return str(response)

if __name__ == '__main__':
    app.run(debug=True)